In [60]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
import spacy
from sklearn.metrics.pairwise import cosine_similarity

import matplotlib.pyplot as plt
from imageio import imread

import warnings
warnings.filterwarnings("ignore")

In [61]:
xdf = pd.read_csv("games.csv")

In [62]:
xdf

,Unnamed: 0,Title,Release Date,Team,Rating,Times Listed,Number of Reviews,Genres,Summary,Reviews,Plays,Playing,Backlogs,Wishlist
0,0,Elden Ring,"Feb 25, 2022","['Bandai Namco Entertainment', 'FromSoftware']",4.5,3.9K,3.9K,"['Adventure', 'RPG']","Elden Ring is a fantasy, action and open world game with RPG elements such as stats, weapons and spells. Rise, Tarnished, and be guided by grace to brandish the power of the Elden Ring and become an Elden Lord in the Lands Between.","[""The first playthrough of elden ring is one of the best eperiences gaming can offer you but after youve explored everything in the open world and you've experienced all of the surprises you lose motivation to go exploring on repeat playthroughs which takes a lot away from the replayability which is a very important thing for from games imo."", 'a replay solidified my love for elden ring. so easily my favorite game of all time. actually beating malenia this time was also an amazing feeling. i just love being in this world man its the greatest of all time', 'The game is absolutely beautiful, with so much to do. The replayability is crazy. And it never gets old with it too.', 'Took everything great about the Soulsborne games and make it 100% better.', 'I play with my overlevelled friend every time and we still fail sometimes (he’s on NG6), insanely difficult game lol\n \n gorgeous graphics, animations, everything about this game is so beautiful. deserves all the hype it gets. If you have a powerful enough pc I definitely recommend you play it on there, if you use keyboard, you’ll want to change the keybinds because the defaults are ass.', ""Tied as my favorite souls game with Bloodborne, truly one I'll keep coming back too.""]",17K,3.8K,4.6K,4.8K
1,1,Hades,"Dec 10, 2019",['Supergiant Games'],4.3,2.9K,2.9K,"['Adventure', 'Brawler', 'Indie', 'RPG']","A rogue-lite hack and slash dungeon crawler in which Zagreus, son of Hades the Greek god of the dead, attempts to escape his home and his oppressive father by fighting the souls of the dead through the various layers of the ever-shifting underworld, while getting to know and forging relationships with its inhabitants.","['convinced this is a roguelike for people who do not like the genre. The art is technically good but the aesthetic does very little for me, too clean or something. idk wish I liked it but I do not', 'incredible art, a banger soundtrack a surprisingly charming storyline and a big red dog. hades has very smooth gameplay but after playing it for a while it just feels like youre doing the same 3 hit combo ove and over again, which dont get me wrong that combo feels great to do but once i becomes muscle memory you kinda go into autopilot. also i think the boons are cool but i would like to see them change the flow of combat a bit more, there are some cool ones like the grape boon that can change the way you play but most of them are just upgrades that dont really change the gameplay or strategy. other then those 2 small gripes the game is fantastic.', ""People tell me this game gets really really good at some point but i've beaten entire games in the amount of time I gave this game."", 'Não sou muito de jogo indie, admito que joguei poucos, mas devo dizer que Hades foi um dos melhores jogos que joguei na vida, facilmente no top 15. Toda sua arte, dublagem, gameplay, carisma dos personagens, é tudo incrivel', ""I can't shake the feeling that I am thoroughly bored playing Hades. I've picked up Hades three times over the last few years, and three times I've found myself putting down the controller after a handful of hours, especially after reaching the third stage, and I couldn't figure out why it wasn't working for me. After giving it a few good goes, I've finally managed to pin down what it is that just doesn't click for me about Hades."", 'One of my favorites in the rogue-likes/lites, only being beat by binding of Isaac. I love the characters (Thanatos my beloved) and the writing and the art style and

In [63]:
xdf.head()

,Unnamed: 0,Title,Release Date,Team,Rating,Times Listed,Number of Reviews,Genres,Summary,Reviews,Plays,Playing,Backlogs,Wishlist
0,0,Elden Ring,"Feb 25, 2022","['Bandai Namco Entertainment', 'FromSoftware']",4.5,3.9K,3.9K,"['Adventure', 'RPG']","Elden Ring is a fantasy, action and open world game with RPG elements such as stats, weapons and spells. Rise, Tarnished, and be guided by grace to brandish the power of the Elden Ring and become an Elden Lord in the Lands Between.","[""The first playthrough of elden ring is one of the best eperiences gaming can offer you but after youve explored everything in the open world and you've experienced all of the surprises you lose motivation to go exploring on repeat playthroughs which takes a lot away from the replayability which is a very important thing for from games imo."", 'a replay solidified my love for elden ring. so easily my favorite game of all time. actually beating malenia this time was also an amazing feeling. i just love being in this world man its the greatest of all time', 'The game is absolutely beautiful, with so much to do. The replayability is crazy. And it never gets old with it too.', 'Took everything great about the Soulsborne games and make it 100% better.', 'I play with my overlevelled friend every time and we still fail sometimes (he’s on NG6), insanely difficult game lol\n \n gorgeous graphics, animations, everything about this game is so beautiful. deserves all the hype it gets. If you have a powerful enough pc I definitely recommend you play it on there, if you use keyboard, you’ll want to change the keybinds because the defaults are ass.', ""Tied as my favorite souls game with Bloodborne, truly one I'll keep coming back too.""]",17K,3.8K,4.6K,4.8K
1,1,Hades,"Dec 10, 2019",['Supergiant Games'],4.3,2.9K,2.9K,"['Adventure', 'Brawler', 'Indie', 'RPG']","A rogue-lite hack and slash dungeon crawler in which Zagreus, son of Hades the Greek god of the dead, attempts to escape his home and his oppressive father by fighting the souls of the dead through the various layers of the ever-shifting underworld, while getting to know and forging relationships with its inhabitants.","['convinced this is a roguelike for people who do not like the genre. The art is technically good but the aesthetic does very little for me, too clean or something. idk wish I liked it but I do not', 'incredible art, a banger soundtrack a surprisingly charming storyline and a big red dog. hades has very smooth gameplay but after playing it for a while it just feels like youre doing the same 3 hit combo ove and over again, which dont get me wrong that combo feels great to do but once i becomes muscle memory you kinda go into autopilot. also i think the boons are cool but i would like to see them change the flow of combat a bit more, there are some cool ones like the grape boon that can change the way you play but most of them are just upgrades that dont really change the gameplay or strategy. other then those 2 small gripes the game is fantastic.', ""People tell me this game gets really really good at some point but i've beaten entire games in the amount of time I gave this game."", 'Não sou muito de jogo indie, admito que joguei poucos, mas devo dizer que Hades foi um dos melhores jogos que joguei na vida, facilmente no top 15. Toda sua arte, dublagem, gameplay, carisma dos personagens, é tudo incrivel', ""I can't shake the feeling that I am thoroughly bored playing Hades. I've picked up Hades three times over the last few years, and three times I've found myself putting down the controller after a handful of hours, especially after reaching the third stage, and I couldn't figure out why it wasn't working for me. After giving it a few good goes, I've finally managed to pin down what it is that just doesn't click for me about Hades."", 'One of my favorites in the rogue-likes/lites, only being beat by binding of Isaac. I love the characters (Thanatos my beloved) and the writing and the art style and

In [64]:
xdf.shape

(1512, 14)

In [65]:
xdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1512 entries, 0 to 1511
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         1512 non-null   int64  
 1   Title              1512 non-null   object 
 2   Release Date       1512 non-null   object 
 3   Team               1511 non-null   object 
 4   Rating             1499 non-null   float64
 5   Times Listed       1512 non-null   object 
 6   Number of Reviews  1512 non-null   object 
 7   Genres             1512 non-null   object 
 8   Summary            1511 non-null   object 
 9   Reviews            1512 non-null   object 
 10  Plays              1512 non-null   object 
 11  Playing            1512 non-null   object 
 12  Backlogs           1512 non-null   object 
 13  Wishlist           1512 non-null   object 
dtypes: float64(1), int64(1), object(12)
memory usage: 165.5+ KB


In [66]:
dataG = xdf[[
      'Title',
      'Release Date',
      'Genres',
      'Summary',
      'Reviews',
      'Rating',
      'Playing',
      'Backlogs',
      'Wishlist',
      'Plays']]
dataG.head()

,Title,Release Date,Genres,Summary,Reviews,Rating,Playing,Backlogs,Wishlist,Plays
0,Elden Ring,"Feb 25, 2022","['Adventure', 'RPG']","Elden Ring is a fantasy, action and open world game with RPG elements such as stats, weapons and spells. Rise, Tarnished, and be guided by grace to brandish the power of the Elden Ring and become an Elden Lord in the Lands Between.","[""The first playthrough of elden ring is one of the best eperiences gaming can offer you but after youve explored everything in the open world and you've experienced all of the surprises you lose motivation to go exploring on repeat playthroughs which takes a lot away from the replayability which is a very important thing for from games imo."", 'a replay solidified my love for elden ring. so easily my favorite game of all time. actually beating malenia this time was also an amazing feeling. i just love being in this world man its the greatest of all time', 'The game is absolutely beautiful, with so much to do. The replayability is crazy. And it never gets old with it too.', 'Took everything great about the Soulsborne games and make it 100% better.', 'I play with my overlevelled friend every time and we still fail sometimes (he’s on NG6), insanely difficult game lol\n \n gorgeous graphics, animations, everything about this game is so beautiful. deserves all the hype it gets. If you have a powerful enough pc I definitely recommend you play it on there, if you use keyboard, you’ll want to change the keybinds because the defaults are ass.', ""Tied as my favorite souls game with Bloodborne, truly one I'll keep coming back too.""]",4.5,3.8K,4.6K,4.8K,17K
1,Hades,"Dec 10, 2019","['Adventure', 'Brawler', 'Indie', 'RPG']","A rogue-lite hack and slash dungeon crawler in which Zagreus, son of Hades the Greek god of the dead, attempts to escape his home and his oppressive father by fighting the souls of the dead through the various layers of the ever-shifting underworld, while getting to know and forging relationships with its inhabitants.","['convinced this is a roguelike for people who do not like the genre. The art is technically good but the aesthetic does very little for me, too clean or something. idk wish I liked it but I do not', 'incredible art, a banger soundtrack a surprisingly charming storyline and a big red dog. hades has very smooth gameplay but after playing it for a while it just feels like youre doing the same 3 hit combo ove and over again, which dont get me wrong that combo feels great to do but once i becomes muscle memory you kinda go into autopilot. also i think the boons are cool but i would like to see them change the flow of combat a bit more, there are some cool ones like the grape boon that can change the way you play but most of them are just upgrades that dont really change the gameplay or strategy. other then those 2 small gripes the game is fantastic.', ""People tell me this game gets really really good at some point but i've beaten entire games in the amount of time I gave this game."", 'Não sou muito de jogo indie, admito que joguei poucos, mas devo dizer que Hades foi um dos melhores jogos que joguei na vida, facilmente no top 15. Toda sua arte, dublagem, gameplay, carisma dos personagens, é tudo incrivel', ""I can't shake the feeling that I am thoroughly bored playing Hades. I've picked up Hades three times over the last few years, and three times I've found myself putting down the controller after a handful of hours, especially after reaching the third stage, and I couldn't figure out why it wasn't working for me. After giving it a few good goes, I've finally managed to pin down what it is that just doesn't click for me about Hades."", 'One of my favorites in the rogue-likes/lites, only being beat by binding of Isaac. I love the characters (Thanatos my beloved) and the writing and the art style and RRRAAAAHHH THIS GAME IS AWESOME']",4.3,3.2K,6.3K,3.6K,21K
2,The Legend of Zelda: Breath of the Wild,"Mar 03, 2017","['Adventure', 'RPG']","The L

In [67]:
dataG.isnull().sum()

Title            0
Release Date     0
Genres           0
Summary          1
Reviews          0
Rating          13
Playing          0
Backlogs         0
Wishlist         0
Plays            0
dtype: int64

In [68]:
dataG.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1512 entries, 0 to 1511
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Title         1512 non-null   object 
 1   Release Date  1512 non-null   object 
 2   Genres        1512 non-null   object 
 3   Summary       1511 non-null   object 
 4   Reviews       1512 non-null   object 
 5   Rating        1499 non-null   float64
 6   Playing       1512 non-null   object 
 7   Backlogs      1512 non-null   object 
 8   Wishlist      1512 non-null   object 
 9   Plays         1512 non-null   object 
dtypes: float64(1), object(9)
memory usage: 118.3+ KB


In [69]:
#removing rows with NULL value
#for example, games with no reviews

In [70]:
#drop all NULL values
dataG=dataG.dropna()

#reset index rows of dataframe to avoid non-consecutive index values
dataG=dataG.reset_index(drop=True)

print(f"Shape of new dataset is: {dataG.shape}")

Shape of new dataset is: (1499, 10)


In [71]:
#there were 13 rows wil null values
#Double checking if there are any null values left in the dataset
dataG.isnull().values.any()


False

In [72]:
#Extract year from release data and drop Release date column
import datetime

def extract_year(date_str):
    try:
        date_object = datetime.datetime.strptime(date_str, "%b %d, %Y")
        return date_object.year
    except ValueError:
        return pd.NA  # Handle potential format errors (optional)

dataG['Year'] = dataG['Release Date'].apply(extract_year)
#dataG.drop('Release Date',axis=1) 
data=dataG.drop('Release Date',axis=1) 
data.head()

,Title,Genres,Summary,Reviews,Rating,Playing,Backlogs,Wishlist,Plays,Year
0,Elden Ring,"['Adventure', 'RPG']","Elden Ring is a fantasy, action and open world game with RPG elements such as stats, weapons and spells. Rise, Tarnished, and be guided by grace to brandish the power of the Elden Ring and become an Elden Lord in the Lands Between.","[""The first playthrough of elden ring is one of the best eperiences gaming can offer you but after youve explored everything in the open world and you've experienced all of the surprises you lose motivation to go exploring on repeat playthroughs which takes a lot away from the replayability which is a very important thing for from games imo."", 'a replay solidified my love for elden ring. so easily my favorite game of all time. actually beating malenia this time was also an amazing feeling. i just love being in this world man its the greatest of all time', 'The game is absolutely beautiful, with so much to do. The replayability is crazy. And it never gets old with it too.', 'Took everything great about the Soulsborne games and make it 100% better.', 'I play with my overlevelled friend every time and we still fail sometimes (he’s on NG6), insanely difficult game lol\n \n gorgeous graphics, animations, everything about this game is so beautiful. deserves all the hype it gets. If you have a powerful enough pc I definitely recommend you play it on there, if you use keyboard, you’ll want to change the keybinds because the defaults are ass.', ""Tied as my favorite souls game with Bloodborne, truly one I'll keep coming back too.""]",4.5,3.8K,4.6K,4.8K,17K,2022
1,Hades,"['Adventure', 'Brawler', 'Indie', 'RPG']","A rogue-lite hack and slash dungeon crawler in which Zagreus, son of Hades the Greek god of the dead, attempts to escape his home and his oppressive father by fighting the souls of the dead through the various layers of the ever-shifting underworld, while getting to know and forging relationships with its inhabitants.","['convinced this is a roguelike for people who do not like the genre. The art is technically good but the aesthetic does very little for me, too clean or something. idk wish I liked it but I do not', 'incredible art, a banger soundtrack a surprisingly charming storyline and a big red dog. hades has very smooth gameplay but after playing it for a while it just feels like youre doing the same 3 hit combo ove and over again, which dont get me wrong that combo feels great to do but once i becomes muscle memory you kinda go into autopilot. also i think the boons are cool but i would like to see them change the flow of combat a bit more, there are some cool ones like the grape boon that can change the way you play but most of them are just upgrades that dont really change the gameplay or strategy. other then those 2 small gripes the game is fantastic.', ""People tell me this game gets really really good at some point but i've beaten entire games in the amount of time I gave this game."", 'Não sou muito de jogo indie, admito que joguei poucos, mas devo dizer que Hades foi um dos melhores jogos que joguei na vida, facilmente no top 15. Toda sua arte, dublagem, gameplay, carisma dos personagens, é tudo incrivel', ""I can't shake the feeling that I am thoroughly bored playing Hades. I've picked up Hades three times over the last few years, and three times I've found myself putting down the controller after a handful of hours, especially after reaching the third stage, and I couldn't figure out why it wasn't working for me. After giving it a few good goes, I've finally managed to pin down what it is that just doesn't click for me about Hades."", 'One of my favorites in the rogue-likes/lites, only being beat by binding of Isaac. I love the characters (Thanatos my beloved) and the writing and the art style and RRRAAAAHHH THIS GAME IS AWESOME']",4.3,3.2K,6.3K,3.6K,21K,2019
2,The Legend of Zelda: Breath of the Wild,"['Adventure', 'RPG']","The Legend of Zelda: Breath of the Wild is the f

In [73]:
data.isnull().values.any()

True

In [74]:
#Release date column contain strings with "releases on TBD" which means the game has not been released yet
#extracting the year from this column cause this string value to be null and now we also have to remove the rows containing it
#drop all NULL values
data=data.dropna()

#reset index rows of datframe to avoid non-consecutive index values
data=data.reset_index(drop=True)

#convert date column from string to int
data['Year']=data['Year'].astype(int)
#df['points'] = df['points'].astype(str).astype(int)

print(f"Shape of new dataset is: {data.shape}")


Shape of new dataset is: (1497, 10)


In [75]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1497 entries, 0 to 1496
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Title     1497 non-null   object 
 1   Genres    1497 non-null   object 
 2   Summary   1497 non-null   object 
 3   Reviews   1497 non-null   object 
 4   Rating    1497 non-null   float64
 5   Playing   1497 non-null   object 
 6   Backlogs  1497 non-null   object 
 7   Wishlist  1497 non-null   object 
 8   Plays     1497 non-null   object 
 9   Year      1497 non-null   int32  
dtypes: float64(1), int32(1), object(8)
memory usage: 111.2+ KB


In [76]:
# Converts the numbers with K to multiple by 1000 and convert into whole integers since K = 1000. Eg. 1K = 1 * 1000
def extract_value(value_str):
    if value_str.upper().endswith('K'):
        value_str = value_str[:-1]
        value_num = float(value_str) * 1000
    else:
        value_num = float(value_str)
    return value_num


In [77]:
games_clean = data.copy()

#replaces K for 1000
games_clean["Backlogs"] = games_clean["Backlogs"].str.replace("K","000")
games_clean["Backlogs"] = games_clean["Backlogs"].str.replace(".","")
games_clean["Wishlist"] = games_clean["Wishlist"].str.replace("K","000")
games_clean["Wishlist"] = games_clean["Wishlist"].str.replace(".","")

games_clean[['Backlogs', 'Wishlist']] = games_clean[['Backlogs', 'Wishlist']].apply(pd.to_numeric)

#combines column backlogs and wishlist
#for people who are interested
games_clean['Interested'] = games_clean["Backlogs"] + games_clean["Wishlist"]
games_clean['Currently Playing']=games_clean['Playing'].apply(extract_value)
games_clean['Total Plays']=games_clean['Plays'].apply(extract_value)

#convert to float (except year) for consistency
games_clean['Currently Playing']=games_clean['Currently Playing'].astype(float)
games_clean['Interested']=games_clean['Interested'].astype(float)
games_clean['Total Plays']=games_clean['Total Plays'].astype(float)

#removes Playing, Backlogs, and Wishlist from df
games_clean.pop("Backlogs")
games_clean.pop("Wishlist")
games_clean.pop("Playing")
games_clean.pop("Plays")


games_clean.head()

,Title,Genres,Summary,Reviews,Rating,Year,Interested,Currently Playing,Total Plays
0,Elden Ring,"['Adventure', 'RPG']","Elden Ring is a fantasy, action and open world game with RPG elements such as stats, weapons and spells. Rise, Tarnished, and be guided by grace to brandish the power of the Elden Ring and become an Elden Lord in the Lands Between.","[""The first playthrough of elden ring is one of the best eperiences gaming can offer you but after youve explored everything in the open world and you've experienced all of the surprises you lose motivation to go exploring on repeat playthroughs which takes a lot away from the replayability which is a very important thing for from games imo."", 'a replay solidified my love for elden ring. so easily my favorite game of all time. actually beating malenia this time was also an amazing feeling. i just love being in this world man its the greatest of all time', 'The game is absolutely beautiful, with so much to do. The replayability is crazy. And it never gets old with it too.', 'Took everything great about the Soulsborne games and make it 100% better.', 'I play with my overlevelled friend every time and we still fail sometimes (he’s on NG6), insanely difficult game lol\n \n gorgeous graphics, animations, everything about this game is so beautiful. deserves all the hype it gets. If you have a powerful enough pc I definitely recommend you play it on there, if you use keyboard, you’ll want to change the keybinds because the defaults are ass.', ""Tied as my favorite souls game with Bloodborne, truly one I'll keep coming back too.""]",4.5,2022,94000.0,3800.0,17000.0
1,Hades,"['Adventure', 'Brawler', 'Indie', 'RPG']","A rogue-lite hack and slash dungeon crawler in which Zagreus, son of Hades the Greek god of the dead, attempts to escape his home and his oppressive father by fighting the souls of the dead through the various layers of the ever-shifting underworld, while getting to know and forging relationships with its inhabitants.","['convinced this is a roguelike for people who do not like the genre. The art is technically good but the aesthetic does very little for me, too clean or something. idk wish I liked it but I do not', 'incredible art, a banger soundtrack a surprisingly charming storyline and a big red dog. hades has very smooth gameplay but after playing it for a while it just feels like youre doing the same 3 hit combo ove and over again, which dont get me wrong that combo feels great to do but once i becomes muscle memory you kinda go into autopilot. also i think the boons are cool but i would like to see them change the flow of combat a bit more, there are some cool ones like the grape boon that can change the way you play but most of them are just upgrades that dont really change the gameplay or strategy. other then those 2 small gripes the game is fantastic.', ""People tell me this game gets really really good at some point but i've beaten entire games in the amount of time I gave this game."", 'Não sou muito de jogo indie, admito que joguei poucos, mas devo dizer que Hades foi um dos melhores jogos que joguei na vida, facilmente no top 15. Toda sua arte, dublagem, gameplay, carisma dos personagens, é tudo incrivel', ""I can't shake the feeling that I am thoroughly bored playing Hades. I've picked up Hades three times over the last few years, and three times I've found myself putting down the controller after a handful of hours, especially after reaching the third stage, and I couldn't figure out why it wasn't working for me. After giving it a few good goes, I've finally managed to pin down what it is that just doesn't click for me about Hades."", 'One of my favorites in the rogue-likes/lites, only being beat by binding of Isaac. I love the characters (Thanatos my beloved) and the writing and the art style and RRRAAAAHHH THIS GAME IS AWESOME']",4.3,2019,99000.0,3200.0,21000.0
2,The Legend of Zelda: Breath of the Wild,"['Adventure', 'RPG']","The Legend of Zelda: Breath of 

In [78]:
# Check if there are any other NULL values
games_clean.info()
games_clean.isnull().values.any()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1497 entries, 0 to 1496
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Title              1497 non-null   object 
 1   Genres             1497 non-null   object 
 2   Summary            1497 non-null   object 
 3   Reviews            1497 non-null   object 
 4   Rating             1497 non-null   float64
 5   Year               1497 non-null   int32  
 6   Interested         1497 non-null   float64
 7   Currently Playing  1497 non-null   float64
 8   Total Plays        1497 non-null   float64
dtypes: float64(4), int32(1), object(4)
memory usage: 99.5+ KB


False

In [79]:
#Remove brackets and ' in Genres. 
#check type if index 0 exist
print (type(games_clean.loc[0, 'Genres']))


#check type generally, index can be `DatetimeIndex`, `FloatIndex`...
print (type(games_clean.loc[games_clean.index[0], 'Genres']))
games_clean['Genres'] = games_clean['Genres'].str.strip('[]')
games_clean['Genres'] = games_clean['Genres'].str.replace(r"' '", " ", regex=True)

games_clean.head()

<class 'str'>
<class 'str'>


,Title,Genres,Summary,Reviews,Rating,Year,Interested,Currently Playing,Total Plays
0,Elden Ring,"'Adventure', 'RPG'","Elden Ring is a fantasy, action and open world game with RPG elements such as stats, weapons and spells. Rise, Tarnished, and be guided by grace to brandish the power of the Elden Ring and become an Elden Lord in the Lands Between.","[""The first playthrough of elden ring is one of the best eperiences gaming can offer you but after youve explored everything in the open world and you've experienced all of the surprises you lose motivation to go exploring on repeat playthroughs which takes a lot away from the replayability which is a very important thing for from games imo."", 'a replay solidified my love for elden ring. so easily my favorite game of all time. actually beating malenia this time was also an amazing feeling. i just love being in this world man its the greatest of all time', 'The game is absolutely beautiful, with so much to do. The replayability is crazy. And it never gets old with it too.', 'Took everything great about the Soulsborne games and make it 100% better.', 'I play with my overlevelled friend every time and we still fail sometimes (he’s on NG6), insanely difficult game lol\n \n gorgeous graphics, animations, everything about this game is so beautiful. deserves all the hype it gets. If you have a powerful enough pc I definitely recommend you play it on there, if you use keyboard, you’ll want to change the keybinds because the defaults are ass.', ""Tied as my favorite souls game with Bloodborne, truly one I'll keep coming back too.""]",4.5,2022,94000.0,3800.0,17000.0
1,Hades,"'Adventure', 'Brawler', 'Indie', 'RPG'","A rogue-lite hack and slash dungeon crawler in which Zagreus, son of Hades the Greek god of the dead, attempts to escape his home and his oppressive father by fighting the souls of the dead through the various layers of the ever-shifting underworld, while getting to know and forging relationships with its inhabitants.","['convinced this is a roguelike for people who do not like the genre. The art is technically good but the aesthetic does very little for me, too clean or something. idk wish I liked it but I do not', 'incredible art, a banger soundtrack a surprisingly charming storyline and a big red dog. hades has very smooth gameplay but after playing it for a while it just feels like youre doing the same 3 hit combo ove and over again, which dont get me wrong that combo feels great to do but once i becomes muscle memory you kinda go into autopilot. also i think the boons are cool but i would like to see them change the flow of combat a bit more, there are some cool ones like the grape boon that can change the way you play but most of them are just upgrades that dont really change the gameplay or strategy. other then those 2 small gripes the game is fantastic.', ""People tell me this game gets really really good at some point but i've beaten entire games in the amount of time I gave this game."", 'Não sou muito de jogo indie, admito que joguei poucos, mas devo dizer que Hades foi um dos melhores jogos que joguei na vida, facilmente no top 15. Toda sua arte, dublagem, gameplay, carisma dos personagens, é tudo incrivel', ""I can't shake the feeling that I am thoroughly bored playing Hades. I've picked up Hades three times over the last few years, and three times I've found myself putting down the controller after a handful of hours, especially after reaching the third stage, and I couldn't figure out why it wasn't working for me. After giving it a few good goes, I've finally managed to pin down what it is that just doesn't click for me about Hades."", 'One of my favorites in the rogue-likes/lites, only being beat by binding of Isaac. I love the characters (Thanatos my beloved) and the writing and the art style and RRRAAAAHHH THIS GAME IS AWESOME']",4.3,2019,99000.0,3200.0,21000.0
2,The Legend of Zelda: Breath of the Wild,"'Adventure', 'RPG'","The Legend of Zelda: Breath of the Wi

In [80]:
games_clean.describe() #not useful, will take out later

,Rating,Year,Interested,Currently Playing,Total Plays
count,1497.000000,1497.000000,1497.000000,1497.000000,1497.000000
mean,3.718236,2012.062124,16187.757515,269.981296,6315.252505
std,0.532018,8.491472,21032.136839,427.783884,5891.900341
min,0.700000,1980.000000,9.000000,0.000000,8.000000
25%,3.400000,2007.000000,715.000000,44.000000,1900.000000
50%,3.800000,2014.000000,2799.000000,115.000000,4300.000000
75%,4.100000,2019.000000,25000.000000,302.000000,9100.000000
max,4.600000,2023.000000,106000.000000,3800.000000,33000.000000


In [81]:
#Check if there are any empty info in the data.
games_clean.isna().sum()

Title                0
Genres               0
Summary              0
Reviews              0
Rating               0
Year                 0
Interested           0
Currently Playing    0
Total Plays          0
dtype: int64

# NLP Part
# Markdowns follow the code below the markdowns

Cleaning text from random unicode characters (In this case is the []), and keeping only letters, numbers, spaces. Applies to summary and reviews

In [82]:
def decode(column):
    column = column.str.decode("unicode_escape")\
    .str.replace(r'[^a-zA-Z1-9\ ]', '', regex=True).str.strip()
    return column

In [83]:
games_clean.loc[:,"Summary"] = decode(games_clean["Summary"])
games_clean.loc[:,"Reviews"] = decode(games_clean["Reviews"])

This part of the code removes ONLY completely non-english reviews, basically games with reviews that does not contain english. Games with english reviews will still stay.
It detects which reviews are in English. Puts the english reviews in a new dataframe called english_reviews.

Why not remove the Spanish Words? 

The sentiment values are still valid. As long as the review contains english words, the sentiment analysis will still use those english words and evaluate whether the reviews are positive or negative. From some games (Eg. 1482, Dragon Ball GT: Final Bout), the sentiment values were below 0, indicating a negative review. Although these reviews contained spanish words, the sentiment values still correleated to the original feels to the game, as there were negative english reveiews in these reviews. Hence, not much of a different can be seen from before and after.

Hence, not much of a different can be seen from before and after.

In [84]:
# Function to detect the language of a review using langid with confidence threshold
def detect_language_with_langdetect(line): 
    from langdetect import detect_langs
    try: 
        langs = detect_langs(line) 
        for item in langs: 
            # The first one returned is usually the one that has the highest probability
            return item.lang, item.prob 
    except: return "err", 0.0 

# Apply language detection to each review and add a 'IsEnglish' column to the DataFrame
games_clean['LanguageDetection'] = games_clean['Reviews'].apply(detect_language_with_langdetect)

#Filter out non-English reviews based on the threshold
english_reviews = games_clean[games_clean['LanguageDetection'].apply(lambda x: x[0] == 'en' and x[1] >= 0.8)]
games_clean = english_reviews

# Output the results with language and confidence percentages
# for idx, (lang, confidence) in enumerate(english_reviews['LanguageDetection']):
#     print(f"Review {idx + 1}: Language={lang}, Confidence={confidence:.2%}")

# print(games_clean)

Writes the top 10 summary and reviews into a text file for easy viewing.

In [85]:
# Assuming 'Reviews' and 'Summary' are columns in the DataFrame
top_reviews = games_clean[['Reviews', 'Summary']].head(10)

# Output to a text file
top_reviews.to_csv('reviews_and_summary.txt', sep='\t', index=False)

Resets the index after cleaning of random unicode characters is done to reorganise dataset.

In [86]:
games_clean.reset_index(inplace=True, drop=True)
games_clean.head()

,Title,Genres,Summary,Reviews,Rating,Year,Interested,Currently Playing,Total Plays,LanguageDetection
0,Elden Ring,"'Adventure', 'RPG'",Elden Ring is a fantasy action and open world game with RPG elements such as stats weapons and spells Rise Tarnished and be guided by grace to brandish the power of the Elden Ring and become an Elden Lord in the Lands Between,The first playthrough of elden ring is one of the best eperiences gaming can offer you but after youve explored everything in the open world and youve experienced all of the surprises you lose motivation to go exploring on repeat playthroughs which takes a lot away from the replayability which is a very important thing for from games imo a replay solidified my love for elden ring so easily my favorite game of all time actually beating malenia this time was also an amazing feeling i just love being in this world man its the greatest of all time The game is absolutely beautiful with so much to do The replayability is crazy And it never gets old with it too Took everything great about the Soulsborne games and make it 1 better I play with my overlevelled friend every time and we still fail sometimes hes on NG6 insanely difficult game lol gorgeous graphics animations everything about this game is so beautiful deserves all the hype it gets If you have a powerful enough pc I definitely recommend you play it on there if you use keyboard youll want to change the keybinds because the defaults are ass Tied as my favorite souls game with Bloodborne truly one Ill keep coming back too,4.5,2022,94000.0,3800.0,17000.0,"(en, 0.9999984598043945)"
1,Hades,"'Adventure', 'Brawler', 'Indie', 'RPG'",A roguelite hack and slash dungeon crawler in which Zagreus son of Hades the Greek god of the dead attempts to escape his home and his oppressive father by fighting the souls of the dead through the various layers of the evershifting underworld while getting to know and forging relationships with its inhabitants,convinced this is a roguelike for people who do not like the genre The art is technically good but the aesthetic does very little for me too clean or something idk wish I liked it but I do not incredible art a banger soundtrack a surprisingly charming storyline and a big red dog hades has very smooth gameplay but after playing it for a while it just feels like youre doing the same 3 hit combo ove and over again which dont get me wrong that combo feels great to do but once i becomes muscle memory you kinda go into autopilot also i think the boons are cool but i would like to see them change the flow of combat a bit more there are some cool ones like the grape boon that can change the way you play but most of them are just upgrades that dont really change the gameplay or strategy other then those 2 small gripes the game is fantastic People tell me this game gets really really good at some point but ive beaten entire games in the amount of time I gave this game No sou muito de jogo indie admito que joguei poucos mas devo dizer que Hades foi um dos melhores jogos que joguei na vida facilmente no top 15 Toda sua arte dublagem gameplay carisma dos personagens tudo incrivel I cant shake the feeling that I am thoroughly bored playing Hades Ive picked up Hades three times over the last few years and three times Ive found myself putting down the controller after a handful of hours especially after reaching the third stage and I couldnt figure out why it wasnt working for me After giving it a few good goes Ive finally managed to pin down what it is that just doesnt click for me about Hades One of my favorites in the roguelikeslites only being beat by binding of Isaac I love the characters Thanatos my beloved and the writing and the art style and RRRAAAAHHH THIS GAME IS AWESOME,4.3,2019,99000.0,3200.0,21000.0,"(en, 0.9999984451762782)"
2,Undertale,"'Adventure', 'Indie', 'RPG', 'Turn Based Strategy'",A small child falls into the Underground where monsters have long been banished by humans and are hunting 

Processes incoming text data using spacy to determine the base form of each word in summary & reviews, then returns a list of lemmatized words.  
Lemmatized words are one single form word.
Eg.  "builds", "building",or "built" to the lemma "build"

This process allows words to be broken down to one singluar form to identify similarities easily and create more accurate results.

This will be used for tf-idf matrix generating later.

In [87]:
nlp = spacy.load('en_core_web_sm')

def lemmatize(text):
    doc = nlp(text)
    tokens = [token for token in doc if not token.is_punct]
    lemmas = [token.lemma_ if token.pos_ != 'PRON' else token.orth_ for token in tokens]
    return lemmas

# games_clean['Reviews'] = games_clean['Reviews'].apply(lemmatize)

TextBlob is a Python library that is used to process textual data. In this project, we are using TextBlob to implement specifically Sentiment Analysis

Here, we are using TextBlob to indicate whether the reviews in the dataset lean towards positive or negative. 
If a sentiment value is above 0, it means it is a positive review.
If it's below 0, it indicates a negative value.
The range of sentiment value is [-1, 1]. The value that is closer to -1 or 1 indicates stronger positive/negative sentiments.

In [88]:
from textblob import TextBlob

# Function to calculate sentiment polarity using TextBlob
def get_sentiment(review):
    blob = TextBlob(review)
    return blob.sentiment.polarity

# Apply sentiment analysis to each review and add a 'Sentiment' column to the DataFrame
games_clean['Sentiment'] = games_clean['Reviews'].apply(get_sentiment)

# Filter positive reviews based on sentiment polarity
positive_reviews = games_clean[games_clean['Sentiment'] > 0]['Reviews']

# Filter negative reviews based on sentiment polarity
negative_reviews = games_clean[games_clean['Sentiment'] < 0]['Reviews']

Converts text data to TF-IDF vectors.
 
TF-IDF is a measure that "evaluates how relevant a word is to a document in a collection of documents". It increases proportionally to the number of times a word appears in a document, but is offset by the number of documents that contain the word.
Example: Words that are common in every document, such as this, what, and if, are rank low even though they may appear many times, since they don’t mean much to that document.

This code creates a matrix with TF-IDF weights for each word. 
Obtains a list of feature names that represent which words are included in these vectors. 

Instead of comparing multiple datasets, will try comparing between reviews and summary to determine the relevancy of the words. From there, can roughly determine which words are relevant and appear very frequently, giving us a gauge of what kind of games people usually prefer and what kind of audience the games generally cater to.

Now let's do it for reviews. Here we are using only positive reviews to get a gauge of what kind of games people usually look out for


In [89]:
tfidf_positive = TfidfVectorizer(stop_words="english", tokenizer=lemmatize) #Reduces common words like 'and', 'the' etc as it is not significant.
tfidf_matrix_positive = tfidf_positive.fit_transform(positive_reviews)
feature_names_positive = tfidf_positive.get_feature_names_out()

In [90]:
# Calculate TF-IDF scores for each word in the positive reviews.
tfidf_scores_positive = tfidf_matrix_positive.sum(axis=0)

# Create a DataFrame to display common words and their TF-IDF scores in positive reviews.
common_words_positive = pd.DataFrame(tfidf_scores_positive, columns=feature_names_positive).transpose()
common_words_positive.columns = ['TF-IDF Score']

# Filter out empty strings and whitespace characters.
common_words_positive = common_words_positive[common_words_positive.index.str.strip() != '']

# Filter out specifc number of letters from words. 4 because 1-4 words usually consists of generic words that are redundant.
common_words_positive = common_words_positive[common_words_positive.index.str.len() > 4]

# Sort the DataFrame by TF-IDF score in descending order.
common_words_positive = common_words_positive.sort_values(by='TF-IDF Score', ascending=False)

# Display top common words without blank spaces and single-letter words.
top_common_words_positive = common_words_positive.head(20)
print(top_common_words_positive)

            TF-IDF Score
really         34.651701
story          30.188165
great          27.655811
character      24.802424
gameplay       23.358838
think          21.739794
pretty         20.278119
level          19.973868
world          17.348154
series         16.857333
design         16.799593
mario          16.796759
combat         16.717983
experience     16.276789
thing          14.501720
little         13.801980
enjoy          13.602277
amazing        13.576012
favorite       13.347468
original       12.855110


Below shows the most common words found in the negatively reviewed games.

In [91]:
tfidf_negative = TfidfVectorizer(stop_words="english", tokenizer=lemmatize) #Reduces common words like 'and', 'the' etc as it is not significant.
tfidf_matrix_negative = tfidf_negative.fit_transform(negative_reviews)
feature_names_negative = tfidf_negative.get_feature_names_out()

non_empty_feature_negative = [i for i, name in enumerate(feature_names_negative) if name.strip() != '']

In [92]:
# Calculate TF-IDF scores for each word in the positive reviews.
tfidf_scores_negative = tfidf_matrix_negative.sum(axis=0)

# Create a DataFrame to display common words and their TF-IDF scores in positive reviews.
common_words_negative = pd.DataFrame(tfidf_scores_negative, columns=feature_names_negative).transpose()
common_words_negative.columns = ['TF-IDF Score']

# Filter out empty strings and whitespace characters.
common_words_negative = common_words_negative[common_words_negative.index.str.strip() != '']

# Filter out specifc number of letters from words. 4 because 1-4 words usually consists of generic words that are redundant.
common_words_negative = common_words_negative[common_words_negative.index.str.len() > 4]

# Sort the DataFrame by TF-IDF score in descending order.
common_words_negative = common_words_negative.sort_values(by='TF-IDF Score', ascending=False)

# Display top common words without blank spaces and single-letter words.
top_common_words_negative = common_words_negative.head(20)
print(top_common_words_negative)

           TF-IDF Score
really         7.060074
story          4.528026
gameplay       4.522558
think          4.099966
thing          3.734549
control        3.613303
level          3.588451
series         3.514860
design         3.405866
character      3.327702
smash          3.253072
friend         3.223354
fucking        3.188881
sonic          3.130055
pretty         3.073649
kinda          3.044704
little         3.013418
enjoy          2.962279
actually       2.818825
mechanic       2.768626


The code below retrieves the top common words that appear in the summary of the negatively reviewed games.
We decided to do this to show the kind of games the people dislike with more data collected. 

We can gather more information on the description (summary) of the negatively reviewed games instead and see if there are any correlation of words between the negatively reviewed games and their summary. Instead of just using negatively reviewed games, by combining it with its summary, it will give us a more concrete result.

In [93]:
# Filter games with positive reviews.
negative_games = games_clean[games_clean['Sentiment'] < 0]

# Extract Summary text from games with positive reviews
negative_summary_text = negative_games['Summary']

In [94]:
tfidf_negative_summary = TfidfVectorizer(stop_words="english", tokenizer=lemmatize)
tfidf_matrix_negative_summary = tfidf_negative_summary.fit_transform(negative_summary_text)
feature_names_negative_summary = tfidf_negative_summary.get_feature_names_out()

In [95]:
# Calculate TF-IDF scores for each word in the positive summary text
tfidf_scores_negative_summary = tfidf_matrix_negative_summary.sum(axis=0)

# Create a DataFrame to display common words and their TF-IDF scores in positive summary text
common_words_negative_summary = pd.DataFrame(tfidf_scores_negative_summary, columns=feature_names_negative_summary).transpose()
common_words_negative_summary.columns = ['TF-IDF Score']

# Filter out empty strings and whitespace characters (optional)
common_words_negative_summary = common_words_negative_summary[common_words_negative_summary.index.str.strip() != '']

# Filter out specific number of letters from words (optional)
common_words_negative_summary = common_words_negative_summary[common_words_negative_summary.index.str.len() > 4]

# Sort the DataFrame by TF-IDF score
common_words_negative_summary = common_words_negative_summary.sort_values(by='TF-IDF Score', ascending=False)

# Display the top common words in positive summary text without blank spaces and single-letter words
top_common_words_negative_summary = common_words_negative_summary.head(20)
print(top_common_words_negative_summary)

            TF-IDF Score
player          6.607703
world           5.772121
series          5.130289
sonic           4.669613
resident        4.634539
battle          4.524839
character       4.494904
feature         3.964053
smash           3.688433
gameplay        3.564589
enemy           3.549561
friend          3.468533
super           3.350877
experience      3.244106
ability         3.025137
story           2.934642
weapon          2.800203
action          2.702605
mario           2.678901
adventure       2.653133


The code below extracts key words from Summary. The summary selected are from the positive reviews that we have identified previously. This would give a better gauge of what types of games the people are favouring based on the summary of the game description.

In [96]:
# Filter games with positive reviews.
positive_games = games_clean[games_clean['Sentiment'] > 0]

# Extract Summary text from games with positive reviews
positive_summary_text = positive_games['Summary']

In [97]:
tfidf_positive_summary = TfidfVectorizer(stop_words="english", tokenizer=lemmatize)
tfidf_matrix_positive_summary = tfidf_positive_summary.fit_transform(positive_summary_text)
feature_names_positive_summary = tfidf_positive_summary.get_feature_names_out()

In [98]:
non_empty_feature_indices = [i for i, name in enumerate(feature_names_positive_summary) if name.strip() != '']

# Filter the TF-IDF matrix and feature names
tfidf_matrix_filtered = tfidf_matrix_positive_summary[:, non_empty_feature_indices]
feature_names_filtered = [feature_names_positive_summary[i] for i in non_empty_feature_indices]

In [99]:
# Calculate TF-IDF scores for each word in the positive summary text
tfidf_scores_positive_summary = tfidf_matrix_positive_summary.sum(axis=0)

# Create a DataFrame to display common words and their TF-IDF scores in positive summary text
common_words_positive_summary = pd.DataFrame(tfidf_scores_positive_summary, columns=feature_names_positive_summary).transpose()
common_words_positive_summary.columns = ['TF-IDF Score']

# Filter out empty strings and whitespace characters (optional)
common_words_positive_summary = common_words_positive_summary[common_words_positive_summary.index.str.strip() != '']

# Filter out specific number of letters from words (optional)
common_words_positive_summary = common_words_positive_summary[common_words_positive_summary.index.str.len() > 4]

# Sort the DataFrame by TF-IDF score
common_words_positive_summary = common_words_positive_summary.sort_values(by='TF-IDF Score', ascending=False)

# Display the top common words in positive summary text without blank spaces and single-letter words
top_common_words_positive_summary = common_words_positive_summary.head(20)
print(top_common_words_positive_summary)

            TF-IDF Score
player         29.731156
world          27.532210
character      20.206521
story          19.779963
mario          18.492129
series         18.153026
adventure      17.919179
feature        17.854989
pokmon         16.091034
action         15.900210
experience     15.515560
fight          15.207340
battle         14.695277
gameplay       14.558944
explore        13.627336
enemy          12.821490
combat         12.417153
super          12.004756
version        11.785118
power          11.473265


# Conclusion from NLP

Using Sentiment Analysis helped us to separate positively reviewed games and negatively reviewed games so that we get to see a comparison between the two results and understand what the gaming community wants and don't want. Stopwords and lemmatization of words also assisted in ensuring the results were more accurate by filtering out unecessary words that were not relevant and allowing the algorithm to understand better what the lemmatized words mean.

From the positive summary TF-IDF scores, we understand that many players favour games that have good or well-written characters and stories. 

They also seem to focus on the world of the games, perhaps from world-building or how well the world is designed. Mario & Pokemon seems to pop up very often too, showing that people are attracted to the popular long and on-going series.

The words "adventure","action", "fight", "battle", "gameplay", "enemy", "combat" also appeared quite often. These words generally mean that people are looking out for more adventure and action games that have intensive battles/combats with enemies and have really good and fun gameplay.

From the negative summary TF-IDF scores, we see that many players have their own reasons why they dislike certain games. Some words are repeated here that was seen from the positive summary reviews.

Some seem to not like the world of the games, some also don't seem to like certain series like Sonic the Hedgedog & Resident Evil.

Words like "character", "battle", "feature", "gameplay", "experience" and "story" show that some people might not favour certain games because they lack aspect in one of these or more. 

There is a vocal minority that seem to not enjoy adventure and action type of games.

With all of these conclusions, game creators can have a better idea of what elements the community is looking out for in games currently that will captivate them to play and enjoy themselves.